In [1]:
import os
import sys
import random
import time
import argparse
import numpy as np
import matplotlib.pyplot as plt

import multiprocessing as mp

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

from models import DPINet
#from data import PhysicsFleXDataset, collate_fn

from utils import count_parameters

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--pstep', type=int, default=2)
parser.add_argument('--n_rollout', type=int, default=0)
parser.add_argument('--time_step', type=int, default=0)
parser.add_argument('--time_step_clip', type=int, default=0)
parser.add_argument('--dt', type=float, default=1./60.)
parser.add_argument('--nf_relation', type=int, default=300)
parser.add_argument('--nf_particle', type=int, default=200)
parser.add_argument('--nf_effect', type=int, default=200)
parser.add_argument('--env', default='')
parser.add_argument('--train_valid_ratio', type=float, default=0.9)
parser.add_argument('--outf', default='files')
parser.add_argument('--dataf', default='data')
parser.add_argument('--num_workers', type=int, default=10)
parser.add_argument('--gen_data', type=int, default=0)
parser.add_argument('--gen_stat', type=int, default=0)
parser.add_argument('--log_per_iter', type=int, default=1000)
parser.add_argument('--ckp_per_iter', type=int, default=10000)
parser.add_argument('--eval', type=int, default=0)
parser.add_argument('--verbose_data', type=int, default=1)
parser.add_argument('--verbose_model', type=int, default=1)

parser.add_argument('--n_instance', type=int, default=0)
parser.add_argument('--n_stages', type=int, default=0)
parser.add_argument('--n_his', type=int, default=0)

parser.add_argument('--n_epoch', type=int, default=1000)
parser.add_argument('--beta1', type=float, default=0.9)
parser.add_argument('--lr', type=float, default=0.0001)
parser.add_argument('--batch_size', type=int, default=1)
parser.add_argument('--forward_times', type=int, default=2)

parser.add_argument('--resume_epoch', type=int, default=0)
parser.add_argument('--resume_iter', type=int, default=0)

# shape state:
# [x, y, z, x_last, y_last, z_last, quat(4), quat_last(4)]
parser.add_argument('--shape_state_dim', type=int, default=14)

# object attributes:
parser.add_argument('--attr_dim', type=int, default=0)

# object state:
parser.add_argument('--state_dim', type=int, default=0)
parser.add_argument('--position_dim', type=int, default=0)

# relation attr:
parser.add_argument('--relation_dim', type=int, default=0)

_StoreAction(option_strings=['--relation_dim'], dest='relation_dim', nargs=None, const=None, default=0, type=<class 'int'>, choices=None, help=None, metavar=None)

In [3]:
args = parser.parse_args("--env SingleHair --gen_data 1".split())

In [4]:
phases_dict = dict()

args.n_rollout = 50
args.num_workers = 3
args.gen_stat = 1

# object states:
# [x, y, z, xdot, ydot, zdot]
args.state_dim = 6
args.position_dim = 3

# object attr:
# [rigid]
args.attr_dim = 1

# relation attr:
# [none]
args.relation_dim = 1

args.time_step = 500
args.time_step_clip = 20
args.n_instance = 1
args.n_stages = 1

args.neighbor_radius = 0.08

phases_dict["instance_idx"] = [0, 30]
phases_dict["root_num"] = [[]]
phases_dict["instance"] = ['solid']
phases_dict["material"] = ['solid']

args.outf = 'dump_SingleHair/' + args.outf

In [5]:
args.outf = args.outf + '_' + args.env
args.dataf = 'data/' + args.dataf + '_' + args.env
print (args.dataf)
os.system('mkdir -p ' + args.outf)
os.system('mkdir -p ' + args.dataf)

data/data_SingleHair


0

In [6]:
from data import *

In [7]:
def gen_PyFleX(info):

    env, root_num = info['env'], info['root_num']
    thread_idx, data_dir, data_names = info['thread_idx'], info['data_dir'], info['data_names']
    n_rollout, n_instance = info['n_rollout'], info['n_instance']
    time_step, time_step_clip = info['time_step'], info['time_step_clip']
    shape_state_dim, dt = info['shape_state_dim'], info['dt']

    env_idx = info['env_idx'] # =11

    np.random.seed(round(time.time() * 1000 + thread_idx) % 2**32)
    
    stats = [init_stat(3), init_stat(3)]

    import pyflex
    pyflex.init()

    for i in range(n_rollout):

        if i % 10 == 0:
            print("%d / %d" % (i, n_rollout))

        rollout_idx = thread_idx * n_rollout + i
        rollout_dir = os.path.join(data_dir, str(rollout_idx))
        os.system('mkdir -p ' + rollout_dir)
        
        # scene_params: [len(box) at dim x,len(box) at dim y,len(box) at dim z, num_hair per circle, num_circle]
        cap_size = [0.1,1.5]
        N_hairs = 1


        scene_params = np.array(cap_size)

        pyflex.set_scene(env_idx, scene_params, thread_idx)
        n_particles = pyflex.get_n_particles()
        n_shapes = 1
        N_particles_per_hair = int(n_particles/N_hairs)
        idx_begins = np.arange(N_hairs)*N_particles_per_hair
        idx_hairs = [[i,i+N_particles_per_hair-1] for i in idx_begins]

        positions = np.zeros((time_step, n_particles, 3), dtype=np.float32)
        velocities = np.zeros((time_step, n_particles, 3), dtype=np.float32)
        shape_position = np.zeros((time_step, n_shapes, 3), dtype=np.float32)
        shape_velocities = np.zeros((time_step, n_shapes, 3), dtype=np.float32)

        for j in range(time_step_clip):
            p_clip = pyflex.get_positions().reshape(-1, 4)[:, :3]
            shape_p_clip = pyflex.get_shape_states()[:3].reshape(-1,3)
            pyflex.step()

        for j in range(time_step):
            positions[j] = pyflex.get_positions().reshape(-1, 4)[:, :3]
            shape_position[j] = pyflex.get_shape_states()[:3].reshape(-1,3)
        #    shape_prevposition = pyflex.get_shape_states()[3:6].reshape(-1,3)
            if j == 0:
                velocities[j] = (positions[j] - p_clip) / dt
                shape_velocities[j] = (shape_position[j] - shape_p_clip)/dt
            else:
                velocities[j] = (positions[j] - positions[j - 1]) / dt
                shape_velocities[j] = (shape_position[j] - shape_position[j-1])/dt

            pyflex.step()
            data = [positions[j], velocities[j], idx_hairs, shape_position[j], shape_velocities[j]]
            store_data(data_names, data, os.path.join(rollout_dir, str(j) + '.h5'))
        
        # change dtype for more accurate stat calculation
        # only normalize positions and velocities
        datas = [positions.astype(np.float64), velocities.astype(np.float64)]

        for j in range(len(stats)): 
            # here j = 2, refers to positions and velocities
            stat = init_stat(stats[j].shape[0]) 
            # stat= np.zeros((3,3))
            stat[:, 0] = np.mean(datas[j], axis=(0, 1))[:]
            stat[:, 1] = np.std(datas[j], axis=(0, 1))[:]
            stat[:, 2] = datas[j].shape[0] * datas[j].shape[1] # time_step*n_particles
            stats[j] = combine_stat(stats[j], stat)

    pyflex.clean()

    return stats

In [14]:
class PhysicsFleXDataset(Dataset):

    def __init__(self, args, phase, phases_dict, verbose):
        self.args = args
        self.phase = phase
        self.phases_dict = phases_dict
        self.verbose = verbose
        self.data_dir = os.path.join(self.args.dataf, phase)
        self.stat_path = os.path.join(self.args.dataf, 'stat.h5')

        os.system('mkdir -p ' + self.data_dir)

        #    self.data_names = ['positions', 'velocities', 'shape_quats', 'clusters', 'scene_params']
        self.data_names = ['positions', 'velocities','hair_idx','shape_position']

        ratio = self.args.train_valid_ratio
        if phase == 'train':
            self.n_rollout = int(self.args.n_rollout * ratio)
        elif phase == 'valid':
            self.n_rollout = self.args.n_rollout - int(self.args.n_rollout * ratio)
        else:
            raise AssertionError("Unknown phase")

    def __len__(self):
        return self.n_rollout * (self.args.time_step - 1)

    def load_data(self, name):
        self.stat = load_data(self.data_names[:2], self.stat_path)
        for i in range(len(self.stat)):
            self.stat[i] = self.stat[i][-self.args.position_dim:, :]
            # print(self.data_names[i], self.stat[i].shape)

    def gen_data(self, name):
        # if the data hasn't been generated, generate the data
        print("Generating data ... n_rollout=%d, time_step=%d" % (self.n_rollout, self.args.time_step))

        infos = []
        for i in range(self.args.num_workers):
            info = {
                'env': self.args.env,
                'root_num': self.phases_dict['root_num'],
                'thread_idx': i,
                'data_dir': self.data_dir,
                'data_names': self.data_names,
                'n_rollout': self.n_rollout // self.args.num_workers,
                'n_instance': self.args.n_instance,
                'time_step': self.args.time_step,
                'time_step_clip': self.args.time_step_clip,
                'dt': self.args.dt,
                'shape_state_dim': self.args.shape_state_dim}

            info['env_idx'] = 11
            infos.append(info)

        cores = self.args.num_workers
        pool = mp.Pool(processes=cores)
        data = pool.map(gen_PyFleX, infos)

        print("Training data generated, warpping up stats ...")

        if self.phase == 'train' and self.args.gen_stat:
            # positions [x, y, z], velocities[xdot, ydot, zdot]
            self.stat = [init_stat(3), init_stat(3)]
            for i in range(len(data)):
                for j in range(len(self.stat)):
                    self.stat[j] = combine_stat(self.stat[j], data[i][j])
            store_data(self.data_names[:2], self.stat, self.stat_path)
        else:
            print("Loading stat from %s ..." % self.stat_path)
            self.stat = load_data(self.data_names[:2], self.stat_path)

    def __getitem__(self, idx):
        idx_rollout = idx // (self.args.time_step - 1)
        idx_timestep = idx % (self.args.time_step - 1)

        # ignore the first frame for env RiceGrip
        if self.args.env == 'RiceGrip' and idx_timestep == 0:
            idx_timestep = np.random.randint(1, self.args.time_step - 1)

        data_path = os.path.join(self.data_dir, str(idx_rollout), str(idx_timestep) + '.h5')
        data_nxt_path = os.path.join(self.data_dir, str(idx_rollout), str(idx_timestep + 1) + '.h5')

        data = load_data(self.data_names, data_path)

        '''
        vel_his = []
        for i in range(self.args.n_his):
            path = os.path.join(self.data_dir, str(idx_rollout), str(max(1, idx_timestep - i - 1)) + '.h5')
            data_his = load_data(self.data_names, path)
            vel_his.append(data_his[1])

        data[1] = np.concatenate([data[1]] + vel_his, 1)
        '''
        
        attr, state, relations, n_particles, n_shapes = \
                prepare_input(data, self.stat, self.args, self.phases_dict, self.verbose)

        ### label
        data_nxt = normalize(load_data(self.data_names, data_nxt_path), self.stat)

        label = torch.FloatTensor(data_nxt[1][:n_particles])

        return attr, state, relations, n_particles, n_shapes, label

In [63]:
def prepare_input(data, stat, args, phases_dict, verbose=0, var=False):
    '''
    for a single hair
    '''
    positions, velocities, hairs_idx, shape_position = data
    n_shapes = 1
    hairs_idx_begin = [idx[0] for idx in hairs_idx]
    n_particles = positions.shape[0]
    R = 0.1
    
    ### object attributes
    #   dim 10: [rigid, fluid, root_0, root_1, gripper_0, gripper_1, mass_inv,
    #            clusterStiffness, clusterPlasticThreshold, cluasterPlasticCreep]
    #   here we only consider the hairs but not the gripper, attr_dim = 1, attr = 0 for hair, attr = 1 for shapes
    attr = np.zeros((n_particles+n_shapes, args.attr_dim))
    
    ### construct relations
    Rr_idxs = []        # relation receiver idx list
    Rs_idxs = []        # relation sender idx list
    Ras = []            # relation attributes list
    values = []         # relation value list (should be 1)
    node_r_idxs = []    # list of corresponding receiver node idx
    node_s_idxs = []    # list of corresponding sender node idx
    psteps = []         # propagation steps
    
    ##### add env specific graph components
    ### specify for shapes
    rels = []
    vals = []
    
    for i in range(n_shapes):
        attr[n_particles+i, 0] = 1
        dis = np.linalg.norm(positions[:,:2]- shape_position[0,:2],axis = 1)
        nodes_rel = np.nonzero(dis <= R)[0]
        # for relation between hair nodes and a gripper, we note it as 1
        gripper = np.ones(nodes_rel.shape[0], dtype=np.int) * (n_particles+i)
        rels += [np.stack([nodes_rel, gripper, np.ones(nodes_rel.shape[0])], axis=1)]
        vals += [np.ones(nodes_rel.shape[0], dtype=np.int)]
        
    
    ##### add relations between leaf particles
    ## here we only consider the relations in a hair: the relation between a node and the nodes nearby
    ## simple case for one hair, TEMPORARY 2 rels for one link
    nodes_p = np.arange(n_particles-1)
    val = np.linalg.norm(positions[1:]-positions[:-1],axis = 1)
    R1 = np.stack([nodes_p,nodes_p+1, np.zeros(n_particles-1)],axis = 1)
    R2 = np.stack([nodes_p+1,nodes_p, np.zeros(n_particles-1)],axis = 1)
    rels += [np.concatenate([R1,R2],axis = 0)]
    vals += [val,val]
    
    rels = np.concatenate(rels, 0)
    vals = np.concatenate(vals, 0)
    
  #  print (vals.shape)
 #   print (rels.shape)
    
    
    if rels.shape[0] > 0:
        if verbose:
            print("Relations neighbor", rels.shape)
        Rr_idxs.append(torch.LongTensor([rels[:, 0], np.arange(rels.shape[0])]))
        Rs_idxs.append(torch.LongTensor([rels[:, 1], np.arange(rels.shape[0])]))
        # Ra: relation attributes
    #    Ra = np.zeros((rels.shape[0], args.relation_dim))  
        Ra = rels[:,2]
        Ras.append(torch.FloatTensor(Ra))
        # values could be changed
     #   values.append(torch.FloatTensor([1] * rels.shape[0]))
      #  values.append(rels[:,2])
        #### for hairs: values equals to the length of this segment
        values.append(torch.FloatTensor(vals))
        node_r_idxs.append(np.arange(n_particles))
        node_s_idxs.append(np.arange(n_particles + n_shapes))
        psteps.append(args.pstep)
        
        
    if verbose:
        print("Attr shape (after hierarchy building):", attr.shape)
        print("Object attr:", np.sum(attr, axis=0))
        print("Particle attr:", np.sum(attr[:n_particles], axis=0))
        print("Shape attr:", np.sum(attr[n_particles:n_particles+n_shapes], axis=0))
        print("Roots attr:", np.sum(attr[n_particles+n_shapes:], axis=0))
        
        
    ### normalize data
    data = [positions, velocities]
    data = normalize(data, stat, var)
    positions, velocities = data[0], data[1]

    if verbose:
        print("Particle positions stats")
        print(positions.shape)
        print(np.min(positions[:n_particles], 0))
        print(np.max(positions[:n_particles], 0))
        print(np.mean(positions[:n_particles], 0))
        print(np.std(positions[:n_particles], 0))

        show_vel_dim = 6 if args.env == 'RiceGrip' else 3
        print("Velocities stats")
        print(velocities.shape)
        print(np.mean(velocities[:n_particles, :show_vel_dim], 0))
        print(np.std(velocities[:n_particles, :show_vel_dim], 0))
        
    state = torch.FloatTensor(np.concatenate([positions, velocities], axis=1))
    attr = torch.FloatTensor(attr)
    relations = [Rr_idxs, Rs_idxs, values, Ras, node_r_idxs, node_s_idxs, psteps]

    return attr, state, relations, n_particles, n_shapes#, instance_idx


    
    

In [64]:
datasets = {phase: PhysicsFleXDataset(
    args, phase, phases_dict, args.verbose_data) for phase in ['train', 'valid']}


In [65]:
datasets['train'].load_data(args.env)

In [66]:
data = datasets['train'].__getitem__(1)

Relations neighbor (60, 3)
Attr shape (after hierarchy building): (32, 1)
Object attr: [1.]
Particle attr: [0.]
Shape attr: [1.]
Roots attr: [0.]
Particle positions stats
(31, 3)
[ 0.46258657 -1.82854189 -0.21068137]
[ 0.46258657  1.6929597  -0.21068137]
[ 0.46258657 -0.09084632 -0.21068137]
[0.00000000e+00 1.05113194e+00 1.38777878e-16]
Velocities stats
(31, 3)
[-0.08565128  2.97585322  0.01638161]
[6.93889390e-17 1.51725969e+00 1.04083409e-17]


In [67]:
attr, state, rels, n_particles, n_shapes, label = data

In [68]:
rels

[[tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
           18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,  1,  2,  3,  4,  5,  6,
            7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
           25, 26, 27, 28, 29, 30],
          [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
           18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
           36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
           54, 55, 56, 57, 58, 59]])],
 [tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
           19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,  0,  1,  2,  3,  4,  5,
            6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
           24, 25, 26, 27, 28, 29],
          [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
           18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 

In [69]:
dataloaders = {x: torch.utils.data.DataLoader(
    datasets[x], batch_size=args.batch_size,
    shuffle=True if x == 'train' else False,
    num_workers=args.num_workers,
    collate_fn=collate_fn)
    for x in ['train', 'valid']}

In [70]:
phase = 'train'
for i, data in enumerate(dataloaders[phase]):
    break


Relations neighborRelations neighborRelations neighbor  

In [71]:
attr, state, rels, n_particles, n_shapes, label = data
Ra, node_r_idx, node_s_idx, pstep = rels[3], rels[4], rels[5], rels[6]

In [72]:
Rr, Rs = [], []
for j in range(len(rels[0])):
    Rr_idx, Rs_idx, values = rels[0][j], rels[1][j], rels[2][j]
    Rr.append(torch.sparse.FloatTensor(Rr_idx, values, torch.Size([node_r_idx[j].shape[0], Ra[j].size(0)])))
    Rs.append(torch.sparse.FloatTensor(Rs_idx, values, torch.Size([node_s_idx[j].shape[0], Ra[j].size(0)])))


In [75]:
data2 = [attr, state, Rr, Rs, Ra, label]

In [77]:
s = 0
Rrp = Rr[s].t()
Rsp = Rs[s].t()

# receiver_attr, sender_attr
attr_r = attr[node_r_idx[s]]
attr_s = attr[node_s_idx[s]]
attr_r_rel = Rrp.mm(attr_r)
attr_s_rel = Rsp.mm(attr_s)

# receiver_state, sender_state
state_r = state[node_r_idx[s]]
state_s = state[node_s_idx[s]]
state_r_rel = Rrp.mm(state_r)
state_s_rel = Rsp.mm(state_s)
state_diff = state_r_rel - state_s_rel

IndexError: index 31 is out of bounds for dimension 0 with size 31

In [79]:
state

tensor([[ 0.4626,  1.6930, -0.2107, -0.0857, -0.0317,  0.0164],
        [ 0.4626,  1.5715, -0.2107, -0.0857, -0.0620,  0.0164],
        [ 0.4626,  1.4512, -0.2107, -0.0857, -0.1093,  0.0164],
        [ 0.4626,  1.3262, -0.2107, -0.0857, -0.1674,  0.0164],
        [ 0.4626,  1.2027, -0.2107, -0.0857, -0.2191,  0.0164],
        [ 0.4626,  1.0792, -0.2107, -0.0857, -0.2719,  0.0164],
        [ 0.4626,  0.9560, -0.2107, -0.0857, -0.3236,  0.0164],
        [ 0.4626,  0.8331, -0.2107, -0.0857, -0.3747,  0.0164],
        [ 0.4626,  0.7104, -0.2107, -0.0857, -0.4247,  0.0164],
        [ 0.4626,  0.5881, -0.2107, -0.0857, -0.4736,  0.0164],
        [ 0.4626,  0.4662, -0.2107, -0.0857, -0.5213,  0.0164],
        [ 0.4626,  0.3446, -0.2107, -0.0857, -0.5677,  0.0164],
        [ 0.4626,  0.2233, -0.2107, -0.0857, -0.6124,  0.0164],
        [ 0.4626,  0.1024, -0.2107, -0.0857, -0.6556,  0.0164],
        [ 0.4626, -0.0180, -0.2107, -0.0857, -0.6970,  0.0164],
        [ 0.4626, -0.1381, -0.2107, -0.0

In [ ]:
d = gen_PyFleX(info)

In [ ]:
stats, datas = d

In [ ]:
stats

In [ ]:
positions, velocities, hair_idx, shape_position = datas

In [139]:
def prepare_input(data, stat, args, phases_dict, verbose=0, var=False):
    '''
    for a single hair
    '''
    positions, velocities, hairs_idx, shape_position = data
    n_shapes = 1
    hairs_idx_begin = [idx[0] for idx in hairs_idx]
    n_particles = positions.shape[0]
    R = 0.1
    R = 2
    
    ### object attributes
    #   dim 10: [rigid, fluid, root_0, root_1, gripper_0, gripper_1, mass_inv,
    #            clusterStiffness, clusterPlasticThreshold, cluasterPlasticCreep]
    #   here we only consider the hairs but not the gripper, attr_dim = 1, attr = 0 for hair, attr = 1 for shapes
    attr = np.zeros((n_particles+n_shapes, args.attr_dim))
    
    ### construct relations
    Rr_idxs = []        # relation receiver idx list
    Rs_idxs = []        # relation sender idx list
    Ras = []            # relation attributes list
    values = []         # relation value list (should be 1)
    node_r_idxs = []    # list of corresponding receiver node idx
    node_s_idxs = []    # list of corresponding sender node idx
    psteps = []         # propagation steps
    
    ##### add env specific graph components
    ### specify for shapes
    rels = []
    vals = []
    
    for i in range(n_shapes):
        attr[n_particles+i, 0] = 1
        dis = np.linalg.norm(positions[:,:2]- shape_position[0,:2],axis = 1)
        nodes_rel = np.nonzero(dis <= R)[0]
        # for relation between hair nodes and a gripper, we note it as 1
        gripper = np.ones(nodes_rel.shape[0], dtype=np.int) * (n_particles+i)
        rels += [np.stack([nodes_rel, gripper, np.ones(nodes_rel.shape[0])], axis=1)]
        vals += [np.ones(nodes_rel.shape[0], dtype=np.int)]
        
    
    ##### add relations between leaf particles
    ## here we only consider the relations in a hair: the relation between a node and the nodes nearby
    ## simple case for one hair, TEMPORARY 2 rels for one link
    nodes_p = np.arange(n_particles-1)
    val = np.linalg.norm(positions[1:]-positions[:-1],axis = 1)
    R1 = np.stack([nodes_p,nodes_p+1, np.zeros(n_particles-1)],axis = 1)
    R2 = np.stack([nodes_p+1,nodes_p, np.zeros(n_particles-1)],axis = 1)
    rels += [np.concatenate([R1,R2],axis = 0)]
    vals += [val,val]
    
    rels = np.concatenate(rels, 0)
    vals = np.concatenate(vals, 0)
    
  #  print (vals.shape)
 #   print (rels.shape)
    
    
    if rels.shape[0] > 0:
        if verbose:
            print("Relations neighbor", rels.shape)
        Rr_idxs.append(torch.LongTensor([rels[:, 0], np.arange(rels.shape[0])]))
        Rs_idxs.append(torch.LongTensor([rels[:, 1], np.arange(rels.shape[0])]))
        # Ra: relation attributes
    #    Ra = np.zeros((rels.shape[0], args.relation_dim))  
        Ra = rels[:,2]
        Ras.append(torch.FloatTensor(Ra))
        # values could be changed
     #   values.append(torch.FloatTensor([1] * rels.shape[0]))
      #  values.append(rels[:,2])
        #### for hairs: values equals to the length of this segment
        values.append(vals)
        node_r_idxs.append(np.arange(n_particles))
        node_s_idxs.append(np.arange(n_particles + n_shapes))
        psteps.append(args.pstep)
        
        
    if verbose:
        print("Attr shape (after hierarchy building):", attr.shape)
        print("Object attr:", np.sum(attr, axis=0))
        print("Particle attr:", np.sum(attr[:n_particles], axis=0))
        print("Shape attr:", np.sum(attr[n_particles:n_particles+n_shapes], axis=0))
        print("Roots attr:", np.sum(attr[n_particles+n_shapes:], axis=0))
        
        
    ### normalize data
    data = [positions, velocities]
    data = normalize(data, stat, var)
    positions, velocities = data[0], data[1]

    if verbose:
        print("Particle positions stats")
        print(positions.shape)
        print(np.min(positions[:n_particles], 0))
        print(np.max(positions[:n_particles], 0))
        print(np.mean(positions[:n_particles], 0))
        print(np.std(positions[:n_particles], 0))

        show_vel_dim = 6 if args.env == 'RiceGrip' else 3
        print("Velocities stats")
        print(velocities.shape)
        print(np.mean(velocities[:n_particles, :show_vel_dim], 0))
        print(np.std(velocities[:n_particles, :show_vel_dim], 0))
        
    state = torch.FloatTensor(np.concatenate([positions, velocities], axis=1))
    attr = torch.FloatTensor(attr)
    relations = [Rr_idxs, Rs_idxs, values, Ras, node_r_idxs, node_s_idxs, psteps]

    return attr, state, relations, n_particles, n_shapes#, instance_idx


    
    
    
    

In [140]:
attr, state, relations, n_particles, n_shapes = prepare_input(data, stat, args, phases_dict)

In [141]:
Rr_idxs, Rs_idxs, values, Ras, node_r_idxs, node_s_idxs, psteps = relations

In [142]:
Rs_idxs

[tensor([[31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31,
          31, 31, 31, 31, 31, 31, 31, 31, 31, 31,  1,  2,  3,  4,  5,  6,  7,  8,
           9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
          27, 28, 29, 30,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
          14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
         [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
          18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
          36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
          54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
          72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87]])]

In [143]:
values

[array([1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 0.11461496, 0.11352849,
        0.11723447, 0.11598921, 0.11598206, 0.11562109, 0.1153357 ,
        0.11500478, 0.11466742, 0.11431694, 0.1139586 , 0.11359477,
        0.11322808, 0.11286092, 0.11249518, 0.11213303, 0.11177492,
        0.11142218, 0.11107552, 0.11073506, 0.11040092, 0.11007237,
        0.10974967, 0.10943198, 0.10911858, 0.10881019, 0.1085012 ,
        0.10820907, 0.10786438, 0.10772401, 0.11461496, 0.11352849,
        0.11723447, 0.11598921, 0.11598206, 0.11562109, 0.1153357 ,
        0.11500478, 0.11466742, 0.11431694, 0.1139586 , 0.11359477,
        0.11322808, 0.11286092, 0.11249518, 0.11

In [144]:
Ras

[tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])]

In [ ]:
def prepare_input(data, stat, args, phases_dict, verbose=0, var=False):

    # Arrangement:
    # particles, shapes, roots

    if args.env == 'RiceGrip':
        positions, velocities, shape_quats, clusters, scene_params = data
        n_shapes = shape_quats.size(0) if var else shape_quats.shape[0]
    elif args.env == 'FluidShake':
        positions, velocities, shape_quats, scene_params = data
        n_shapes = shape_quats.size(0) if var else shape_quats.shape[0]
        clusters = None
    elif args.env == 'BoxBath':
        positions, velocities, clusters = data
        n_shapes = 0
    elif args.env == 'FluidFall':
        positions, velocities = data
        n_shapes = 0
        clusters = None
    elif args.env == 'Hairs':
        positions, velocities, hairs_idx = data
        n_shapes = 1
        hairs_idx_begin = [idx[0] for idx in hairs_idx]
        clusters = None

    count_nodes = positions.size(0) if var else positions.shape[0]
    n_particles = count_nodes - n_shapes

    if verbose:
        print("positions", positions.shape)
        print("velocities", velocities.shape)

        print("n_particles", n_particles)
        print("n_shapes", n_shapes)
        if args.env == 'RiceGrip' or args.env == 'FluidShake':
            print("shape_quats", shape_quats.shape)

    ### instance idx
    #   instance_idx (n_instance + 1): start idx of instance
    if args.env == 'RiceGrip' or args.env == 'FluidShake':
        instance_idx = [0, n_particles]
    elif args.env == 'Hairs':
        instance_idx = [0, count_nodes]
    else:
        instance_idx = phases_dict["instance_idx"]
    if verbose:
        print("Instance_idx:", instance_idx)


    ### object attributes
    #   dim 10: [rigid, fluid, root_0, root_1, gripper_0, gripper_1, mass_inv,
    #            clusterStiffness, clusterPlasticThreshold, cluasterPlasticCreep]
    attr = np.zeros((count_nodes, args.attr_dim))
    # no need to include mass for now
    # attr[:, 6] = positions[:, -1].data.cpu().numpy() if var else positions[:, -1] # mass_inv
    if args.env == 'RiceGrip':
        # clusterStiffness, clusterPlasticThreshold, cluasterPlasticCreep
        attr[:, -3:] = scene_params[-3:]


    ### construct relations
    Rr_idxs = []        # relation receiver idx list
    Rs_idxs = []        # relation sender idx list
    Ras = []            # relation attributes list
    values = []         # relation value list (should be 1)
    node_r_idxs = []    # list of corresponding receiver node idx
    node_s_idxs = []    # list of corresponding sender node idx
    psteps = []         # propagation steps

    ##### add env specific graph components
    rels = []
    if args.env == 'RiceGrip':
        # nodes = np.arange(n_particles)
        for i in range(n_shapes):
            attr[n_particles + i, 2 + i] = 1

            pos = positions.data.cpu().numpy() if var else positions
            dis = np.linalg.norm(
                pos[:n_particles, 3:6:2] - pos[n_particles + i, 3:6:2], axis=1)
            nodes = np.nonzero(dis < 0.3)[0]

            if verbose:
                visualize_neighbors(positions, positions, 0, nodes)
                print(np.sort(dis)[:10])

            gripper = np.ones(nodes.shape[0], dtype=np.int) * (n_particles + i)
            rels += [np.stack([nodes, gripper, np.ones(nodes.shape[0])], axis=1)]
            
    elif args.env == 'Hairs':
        # TODO: add relations between the hairs and the stick
        attr[:,0] = 1
        pass

    elif args.env == 'FluidShake':
        for i in range(n_shapes):
            attr[n_particles + i, 1 + i] = 1

            pos = positions.data.cpu().numpy() if var else positions
            if i == 0:
                # floor
                dis = pos[:n_particles, 1] - pos[n_particles + i, 1]
            elif i == 1:
                # left
                dis = pos[:n_particles, 0] - pos[n_particles + i, 0]
            elif i == 2:
                # right
                dis = pos[n_particles + i, 0] - pos[:n_particles, 0]
            elif i == 3:
                # back
                dis = pos[:n_particles, 2] - pos[n_particles + i, 2]
            elif i == 4:
                # front
                dis = pos[n_particles + i, 2] - pos[:n_particles, 2]
            else:
                raise AssertionError("more shapes than expected")
            nodes = np.nonzero(dis < 0.1)[0]

            if verbose:
                visualize_neighbors(positions, positions, 0, nodes)
                print(np.sort(dis)[:10])

            wall = np.ones(nodes.shape[0], dtype=np.int) * (n_particles + i)
            rels += [np.stack([nodes, wall, np.ones(nodes.shape[0])], axis=1)]

    if verbose and len(rels) > 0:
        print(np.concatenate(rels, 0).shape)

    ##### add relations between leaf particles
    for i in range(len(instance_idx) - 1):
        st, ed = instance_idx[i], instance_idx[i + 1]

        if verbose:
            print('instance #%d' % i, st, ed)

        if args.env == 'BoxBath':
            if phases_dict['material'][i] == 'rigid':
                attr[st:ed, 0] = 1
                queries = np.arange(st, ed)
                anchors = np.concatenate((np.arange(st), np.arange(ed, n_particles)))
            elif phases_dict['material'][i] == 'fluid':
                attr[st:ed, 1] = 1
                queries = np.arange(st, ed)
                anchors = np.arange(n_particles)
            else:
                raise AssertionError("Unsupported materials")
                
        elif args.env == 'Hairs':
            # TODO: add relations between the hairs and the stick
            pass
            
          #  if ed not in hairs_idx_begin:
             #   queries = np.arange(st, ed)
              #  anchors = np.arange(n_particles)

        elif args.env == 'FluidFall' or args.env == 'RiceGrip' or args.env == 'FluidShake':
            if phases_dict['material'][i] == 'fluid':
                attr[st:ed, 0] = 1
                queries = np.arange(st, ed)
                anchors = np.arange(n_particles)
            else:
                raise AssertionError("Unsupported materials")

        else:
            raise AssertionError("Unsupported materials")

        # st_time = time.time()
        pos = positions
        pos = pos[:, -3:]
        if args.env == 'Hairs':
            #TODO
            pass
        else:
            rels += find_relations_neighbor(pos, queries, anchors, args.neighbor_radius, 2, var)
            # return list of [receiver, sender, relation_type]
        # print("Time on neighbor search", time.time() - st_time)

    if verbose:
        print("Attr shape (after add env specific graph components):", attr.shape)
        print("Object attr:", np.sum(attr, axis=0))

    rels = np.concatenate(rels, 0)
    if rels.shape[0] > 0:
        if verbose:
            print("Relations neighbor", rels.shape)
        Rr_idxs.append(torch.LongTensor([rels[:, 0], np.arange(rels.shape[0])]))
        Rs_idxs.append(torch.LongTensor([rels[:, 1], np.arange(rels.shape[0])]))
        Ra = np.zeros((rels.shape[0], args.relation_dim))
        Ras.append(torch.FloatTensor(Ra))
        values.append(torch.FloatTensor([1] * rels.shape[0]))
        node_r_idxs.append(np.arange(n_particles))
        node_s_idxs.append(np.arange(n_particles + n_shapes))
        psteps.append(args.pstep)

    if verbose:
        print('clusters', clusters)

    # add heirarchical relations per instance
    cnt_clusters = 0
    for i in range(len(instance_idx) - 1):
        st, ed = instance_idx[i], instance_idx[i + 1]
        n_root_level = len(phases_dict["root_num"][i])

        if n_root_level > 0:
            attr, positions, velocities, count_nodes, \
            rels, node_r_idx, node_s_idx, pstep = \
                    make_hierarchy(args.env, attr, positions, velocities, i, st, ed,
                                   phases_dict, count_nodes, clusters[cnt_clusters], verbose, var)

            for j in range(len(rels)):
                if verbose:
                    print("Relation instance", j, rels[j].shape)
                Rr_idxs.append(torch.LongTensor([rels[j][:, 0], np.arange(rels[j].shape[0])]))
                Rs_idxs.append(torch.LongTensor([rels[j][:, 1], np.arange(rels[j].shape[0])]))
                Ra = np.zeros((rels[j].shape[0], args.relation_dim)); Ra[:, 0] = 1
                Ras.append(torch.FloatTensor(Ra))
                values.append(torch.FloatTensor([1] * rels[j].shape[0]))
                node_r_idxs.append(node_r_idx[j])
                node_s_idxs.append(node_s_idx[j])
                psteps.append(pstep[j])

            cnt_clusters += 1

    if verbose:
        if args.env == 'RiceGrip' or args.env == 'FluidShake':
            print("Scene_params:", scene_params)

        print("Attr shape (after hierarchy building):", attr.shape)
        print("Object attr:", np.sum(attr, axis=0))
        print("Particle attr:", np.sum(attr[:n_particles], axis=0))
        print("Shape attr:", np.sum(attr[n_particles:n_particles+n_shapes], axis=0))
        print("Roots attr:", np.sum(attr[n_particles+n_shapes:], axis=0))

    ### normalize data
    data = [positions, velocities]
    data = normalize(data, stat, var)
    positions, velocities = data[0], data[1]

    if verbose:
        print("Particle positions stats")
        print(positions.shape)
        print(np.min(positions[:n_particles], 0))
        print(np.max(positions[:n_particles], 0))
        print(np.mean(positions[:n_particles], 0))
        print(np.std(positions[:n_particles], 0))

        show_vel_dim = 6 if args.env == 'RiceGrip' else 3
        print("Velocities stats")
        print(velocities.shape)
        print(np.mean(velocities[:n_particles, :show_vel_dim], 0))
        print(np.std(velocities[:n_particles, :show_vel_dim], 0))

    if args.env == 'RiceGrip':
        if var:
            quats = torch.cat(
                [Variable(torch.zeros(n_particles, 4).cuda()), shape_quats,
                 Variable(torch.zeros(count_nodes - n_particles - n_shapes, 4).cuda())], 0)
            state = torch.cat([positions, velocities, quats], 1)
        else:
            quat_null = np.array([[0., 0., 0., 0.]])
            quats = np.repeat(quat_null, [count_nodes], axis=0)
            quats[n_particles:n_particles + n_shapes] = shape_quats
            # if args.eval == 0:
            # quats += np.random.randn(quats.shape[0], 4) * 0.05
            state = torch.FloatTensor(np.concatenate([positions, velocities, quats], axis=1))
    else:
        if var:
            state = torch.cat([positions, velocities], 1)
        else:
            state = torch.FloatTensor(np.concatenate([positions, velocities], axis=1))

    if verbose:
        for i in range(count_nodes - 1):
            if np.sum(np.abs(attr[i] - attr[i + 1])) > 1e-6:
                print(i, attr[i], attr[i + 1])

        for i in range(len(Ras)):
            print(i, np.min(node_r_idxs[i]), np.max(node_r_idxs[i]), np.min(node_s_idxs[i]), np.max(node_s_idxs[i]))

    attr = torch.FloatTensor(attr)
    relations = [Rr_idxs, Rs_idxs, values, Ras, node_r_idxs, node_s_idxs, psteps]

    return attr, state, relations, n_particles, n_shapes, instance_idx

In [ ]:
def gen_PyFleX(info):

    env, root_num = info['env'], info['root_num']
    thread_idx, data_dir, data_names = info['thread_idx'], info['data_dir'], info['data_names']
    n_rollout, n_instance = info['n_rollout'], info['n_instance']
    time_step, time_step_clip = info['time_step'], info['time_step_clip']
    shape_state_dim, dt = info['shape_state_dim'], info['dt']

    env_idx = info['env_idx']

    np.random.seed(round(time.time() * 1000 + thread_idx) % 2**32)
    
    stats = [init_stat(3), init_stat(3)]

    import pyflex
    pyflex.init()

    for i in range(n_rollout):

        if i % 10 == 0:
            print("%d / %d" % (i, n_rollout))

        rollout_idx = thread_idx * n_rollout + i
        rollout_dir = os.path.join(data_dir, str(rollout_idx))
        os.system('mkdir -p ' + rollout_dir)
        
        env_idx = 10
        # scene_params: [len(box) at dim x,len(box) at dim y,len(box) at dim z, num_hair per circle, num_circle]
        box_size = [0.1,0.1,1.5]
        N_p_cirlcle = 50
        N_circle = 20
        N_hairs = N_circle*N_p_cirlcle


        scene_params = np.array(box_size + [N_p_cirlcle] + [N_circle])

        pyflex.set_scene(env_idx, scene_params, thread_idx)
        n_particles = pyflex.get_n_particles()
        N_particles_per_hair = int(n_particles/N_hairs)
        idx_begins = np.arange(N_hairs)*N_particles_per_hair
        idx_hairs = [[i,i+N_particles_per_hair-1] for i in idx_begins]

        positions = np.zeros((time_step, n_particles, 3), dtype=np.float32)
        velocities = np.zeros((time_step, n_particles, 3), dtype=np.float32)

        for j in range(time_step_clip):
            p_clip = pyflex.get_positions().reshape(-1, 4)[:, :3]
            pyflex.step()

        for j in range(time_step):
            positions[j] = pyflex.get_positions().reshape(-1, 4)[:, :3]
            if j == 0:
                velocities[j] = (positions[j] - p_clip) / dt
            else:
                velocities[j] = (positions[j] - positions[j - 1]) / dt

            pyflex.step()
            data = [positions[j], velocities[j], idx_hairs]
            store_data(data_names, data, os.path.join(rollout_dir, str(j) + '.h5'))
        
        # change dtype for more accurate stat calculation
        # only normalize positions and velocities
        datas = [positions.astype(np.float64), velocities.astype(np.float64)]

        for j in range(len(stats)): 
            # here j = 2, refers to positions and velocities
            stat = init_stat(stats[j].shape[0]) 
            # stat= np.zeros((3,3))
            stat[:, 0] = np.mean(datas[j], axis=(0, 1))[:]
            stat[:, 1] = np.std(datas[j], axis=(0, 1))[:]
            stat[:, 2] = datas[j].shape[0] * datas[j].shape[1] # time_step*n_particles
            stats[j] = combine_stat(stats[j], stat)

    pyflex.clean()

    return stats

In [ ]:
class PhysicsFleXDataset(Dataset):

    def __init__(self, args, phase, phases_dict, verbose):
        self.args = args
        self.phase = phase
        self.phases_dict = phases_dict
        self.verbose = verbose
        self.data_dir = os.path.join(self.args.dataf, phase)
        self.stat_path = os.path.join(self.args.dataf, 'stat.h5')

        os.system('mkdir -p ' + self.data_dir)

        #    self.data_names = ['positions', 'velocities', 'shape_quats', 'clusters', 'scene_params']
        self.data_names = ['positions', 'velocities','hair_idx']

        ratio = self.args.train_valid_ratio
        if phase == 'train':
            self.n_rollout = int(self.args.n_rollout * ratio)
        elif phase == 'valid':
            self.n_rollout = self.args.n_rollout - int(self.args.n_rollout * ratio)
        else:
            raise AssertionError("Unknown phase")

    def __len__(self):
        return self.n_rollout * (self.args.time_step - 1)

    def load_data(self, name):
        self.stat = load_data(self.data_names[:2], self.stat_path)
        for i in range(len(self.stat)):
            self.stat[i] = self.stat[i][-self.args.position_dim:, :]
            # print(self.data_names[i], self.stat[i].shape)

    def gen_data(self, name):
        # if the data hasn't been generated, generate the data
        print("Generating data ... n_rollout=%d, time_step=%d" % (self.n_rollout, self.args.time_step))

        infos = []
        for i in range(self.args.num_workers):
            info = {
                'env': self.args.env,
                'root_num': self.phases_dict['root_num'],
                'thread_idx': i,
                'data_dir': self.data_dir,
                'data_names': self.data_names,
                'n_rollout': self.n_rollout // self.args.num_workers,
                'n_instance': self.args.n_instance,
                'time_step': self.args.time_step,
                'time_step_clip': self.args.time_step_clip,
                'dt': self.args.dt,
                'shape_state_dim': self.args.shape_state_dim}

            info['env_idx'] = 10
            infos.append(info)

        cores = self.args.num_workers
        pool = mp.Pool(processes=cores)
        data = pool.map(gen_PyFleX, infos)

        print("Training data generated, warpping up stats ...")

        if self.phase == 'train' and self.args.gen_stat:
            # positions [x, y, z], velocities[xdot, ydot, zdot]
            if self.args.env == 'RiceGrip':
                self.stat = [init_stat(6), init_stat(6)]
            else:
                self.stat = [init_stat(3), init_stat(3)]
            for i in range(len(data)):
                for j in range(len(self.stat)):
                    self.stat[j] = combine_stat(self.stat[j], data[i][j])
            store_data(self.data_names[:2], self.stat, self.stat_path)
        else:
            print("Loading stat from %s ..." % self.stat_path)
            self.stat = load_data(self.data_names[:2], self.stat_path)

    def __getitem__(self, idx):
        idx_rollout = idx // (self.args.time_step - 1)
        idx_timestep = idx % (self.args.time_step - 1)

        # ignore the first frame for env RiceGrip
        if self.args.env == 'RiceGrip' and idx_timestep == 0:
            idx_timestep = np.random.randint(1, self.args.time_step - 1)

        data_path = os.path.join(self.data_dir, str(idx_rollout), str(idx_timestep) + '.h5')
        data_nxt_path = os.path.join(self.data_dir, str(idx_rollout), str(idx_timestep + 1) + '.h5')

        data = load_data(self.data_names, data_path)

        vel_his = []
        for i in range(self.args.n_his):
            path = os.path.join(self.data_dir, str(idx_rollout), str(max(1, idx_timestep - i - 1)) + '.h5')
            data_his = load_data(self.data_names, path)
            vel_his.append(data_his[1])

        data[1] = np.concatenate([data[1]] + vel_his, 1)

        attr, state, relations, n_particles, n_shapes, instance_idx = \
                prepare_input(data, self.stat, self.args, self.phases_dict, self.verbose)

        ### label
        data_nxt = normalize(load_data(self.data_names, data_nxt_path), self.stat)

        label = torch.FloatTensor(data_nxt[1][:n_particles])

        return attr, state, relations, n_particles, n_shapes, instance_idx, label

In [ ]:
def prepare_input(data, stat, args, phases_dict, verbose=0, var=False):

    # Arrangement:
    # particles, shapes, roots

    if args.env == 'RiceGrip':
        positions, velocities, shape_quats, clusters, scene_params = data
        n_shapes = shape_quats.size(0) if var else shape_quats.shape[0]
    elif args.env == 'FluidShake':
        positions, velocities, shape_quats, scene_params = data
        n_shapes = shape_quats.size(0) if var else shape_quats.shape[0]
        clusters = None
    elif args.env == 'BoxBath':
        positions, velocities, clusters = data
        n_shapes = 0
    elif args.env == 'FluidFall':
        positions, velocities = data
        n_shapes = 0
        clusters = None
    elif args.env == 'Hairs':
        positions, velocities, hairs_idx = data
        n_shapes = 1
        hairs_idx_begin = [idx[0] for idx in hairs_idx]
        clusters = None

    count_nodes = positions.size(0) if var else positions.shape[0]
    n_particles = count_nodes - n_shapes

    if verbose:
        print("positions", positions.shape)
        print("velocities", velocities.shape)

        print("n_particles", n_particles)
        print("n_shapes", n_shapes)
        if args.env == 'RiceGrip' or args.env == 'FluidShake':
            print("shape_quats", shape_quats.shape)

    ### instance idx
    #   instance_idx (n_instance + 1): start idx of instance
    if args.env == 'RiceGrip' or args.env == 'FluidShake':
        instance_idx = [0, n_particles]
    elif args.env == 'Hairs':
        instance_idx = [0, count_nodes]
    else:
        instance_idx = phases_dict["instance_idx"]
    if verbose:
        print("Instance_idx:", instance_idx)


    ### object attributes
    #   dim 10: [rigid, fluid, root_0, root_1, gripper_0, gripper_1, mass_inv,
    #            clusterStiffness, clusterPlasticThreshold, cluasterPlasticCreep]
    attr = np.zeros((count_nodes, args.attr_dim))
    # no need to include mass for now
    # attr[:, 6] = positions[:, -1].data.cpu().numpy() if var else positions[:, -1] # mass_inv
    if args.env == 'RiceGrip':
        # clusterStiffness, clusterPlasticThreshold, cluasterPlasticCreep
        attr[:, -3:] = scene_params[-3:]


    ### construct relations
    Rr_idxs = []        # relation receiver idx list
    Rs_idxs = []        # relation sender idx list
    Ras = []            # relation attributes list
    values = []         # relation value list (should be 1)
    node_r_idxs = []    # list of corresponding receiver node idx
    node_s_idxs = []    # list of corresponding sender node idx
    psteps = []         # propagation steps

    ##### add env specific graph components
    rels = []
    if args.env == 'RiceGrip':
        # nodes = np.arange(n_particles)
        for i in range(n_shapes):
            attr[n_particles + i, 2 + i] = 1

            pos = positions.data.cpu().numpy() if var else positions
            dis = np.linalg.norm(
                pos[:n_particles, 3:6:2] - pos[n_particles + i, 3:6:2], axis=1)
            nodes = np.nonzero(dis < 0.3)[0]

            if verbose:
                visualize_neighbors(positions, positions, 0, nodes)
                print(np.sort(dis)[:10])

            gripper = np.ones(nodes.shape[0], dtype=np.int) * (n_particles + i)
            rels += [np.stack([nodes, gripper, np.ones(nodes.shape[0])], axis=1)]
            
    elif args.env == 'Hairs':
        # TODO: add relations between the hairs and the stick
        attr[:,0] = 1
        pass

    elif args.env == 'FluidShake':
        for i in range(n_shapes):
            attr[n_particles + i, 1 + i] = 1

            pos = positions.data.cpu().numpy() if var else positions
            if i == 0:
                # floor
                dis = pos[:n_particles, 1] - pos[n_particles + i, 1]
            elif i == 1:
                # left
                dis = pos[:n_particles, 0] - pos[n_particles + i, 0]
            elif i == 2:
                # right
                dis = pos[n_particles + i, 0] - pos[:n_particles, 0]
            elif i == 3:
                # back
                dis = pos[:n_particles, 2] - pos[n_particles + i, 2]
            elif i == 4:
                # front
                dis = pos[n_particles + i, 2] - pos[:n_particles, 2]
            else:
                raise AssertionError("more shapes than expected")
            nodes = np.nonzero(dis < 0.1)[0]

            if verbose:
                visualize_neighbors(positions, positions, 0, nodes)
                print(np.sort(dis)[:10])

            wall = np.ones(nodes.shape[0], dtype=np.int) * (n_particles + i)
            rels += [np.stack([nodes, wall, np.ones(nodes.shape[0])], axis=1)]

    if verbose and len(rels) > 0:
        print(np.concatenate(rels, 0).shape)

    ##### add relations between leaf particles
    for i in range(len(instance_idx) - 1):
        st, ed = instance_idx[i], instance_idx[i + 1]

        if verbose:
            print('instance #%d' % i, st, ed)

        if args.env == 'BoxBath':
            if phases_dict['material'][i] == 'rigid':
                attr[st:ed, 0] = 1
                queries = np.arange(st, ed)
                anchors = np.concatenate((np.arange(st), np.arange(ed, n_particles)))
            elif phases_dict['material'][i] == 'fluid':
                attr[st:ed, 1] = 1
                queries = np.arange(st, ed)
                anchors = np.arange(n_particles)
            else:
                raise AssertionError("Unsupported materials")
                
        elif args.env == 'Hairs':
            # TODO: add relations between the hairs and the stick
            pass
            
          #  if ed not in hairs_idx_begin:
             #   queries = np.arange(st, ed)
              #  anchors = np.arange(n_particles)

        elif args.env == 'FluidFall' or args.env == 'RiceGrip' or args.env == 'FluidShake':
            if phases_dict['material'][i] == 'fluid':
                attr[st:ed, 0] = 1
                queries = np.arange(st, ed)
                anchors = np.arange(n_particles)
            else:
                raise AssertionError("Unsupported materials")

        else:
            raise AssertionError("Unsupported materials")

        # st_time = time.time()
        pos = positions
        pos = pos[:, -3:]
        if args.env == 'Hairs':
            #TODO
            pass
        else:
            rels += find_relations_neighbor(pos, queries, anchors, args.neighbor_radius, 2, var)
            # return list of [receiver, sender, relation_type]
        # print("Time on neighbor search", time.time() - st_time)

    if verbose:
        print("Attr shape (after add env specific graph components):", attr.shape)
        print("Object attr:", np.sum(attr, axis=0))

    rels = np.concatenate(rels, 0)
    if rels.shape[0] > 0:
        if verbose:
            print("Relations neighbor", rels.shape)
        Rr_idxs.append(torch.LongTensor([rels[:, 0], np.arange(rels.shape[0])]))
        Rs_idxs.append(torch.LongTensor([rels[:, 1], np.arange(rels.shape[0])]))
        Ra = np.zeros((rels.shape[0], args.relation_dim))
        Ras.append(torch.FloatTensor(Ra))
        values.append(torch.FloatTensor([1] * rels.shape[0]))
        node_r_idxs.append(np.arange(n_particles))
        node_s_idxs.append(np.arange(n_particles + n_shapes))
        psteps.append(args.pstep)

    if verbose:
        print('clusters', clusters)

    # add heirarchical relations per instance
    cnt_clusters = 0
    for i in range(len(instance_idx) - 1):
        st, ed = instance_idx[i], instance_idx[i + 1]
        n_root_level = len(phases_dict["root_num"][i])

        if n_root_level > 0:
            attr, positions, velocities, count_nodes, \
            rels, node_r_idx, node_s_idx, pstep = \
                    make_hierarchy(args.env, attr, positions, velocities, i, st, ed,
                                   phases_dict, count_nodes, clusters[cnt_clusters], verbose, var)

            for j in range(len(rels)):
                if verbose:
                    print("Relation instance", j, rels[j].shape)
                Rr_idxs.append(torch.LongTensor([rels[j][:, 0], np.arange(rels[j].shape[0])]))
                Rs_idxs.append(torch.LongTensor([rels[j][:, 1], np.arange(rels[j].shape[0])]))
                Ra = np.zeros((rels[j].shape[0], args.relation_dim)); Ra[:, 0] = 1
                Ras.append(torch.FloatTensor(Ra))
                values.append(torch.FloatTensor([1] * rels[j].shape[0]))
                node_r_idxs.append(node_r_idx[j])
                node_s_idxs.append(node_s_idx[j])
                psteps.append(pstep[j])

            cnt_clusters += 1

    if verbose:
        if args.env == 'RiceGrip' or args.env == 'FluidShake':
            print("Scene_params:", scene_params)

        print("Attr shape (after hierarchy building):", attr.shape)
        print("Object attr:", np.sum(attr, axis=0))
        print("Particle attr:", np.sum(attr[:n_particles], axis=0))
        print("Shape attr:", np.sum(attr[n_particles:n_particles+n_shapes], axis=0))
        print("Roots attr:", np.sum(attr[n_particles+n_shapes:], axis=0))

    ### normalize data
    data = [positions, velocities]
    data = normalize(data, stat, var)
    positions, velocities = data[0], data[1]

    if verbose:
        print("Particle positions stats")
        print(positions.shape)
        print(np.min(positions[:n_particles], 0))
        print(np.max(positions[:n_particles], 0))
        print(np.mean(positions[:n_particles], 0))
        print(np.std(positions[:n_particles], 0))

        show_vel_dim = 6 if args.env == 'RiceGrip' else 3
        print("Velocities stats")
        print(velocities.shape)
        print(np.mean(velocities[:n_particles, :show_vel_dim], 0))
        print(np.std(velocities[:n_particles, :show_vel_dim], 0))

    if args.env == 'RiceGrip':
        if var:
            quats = torch.cat(
                [Variable(torch.zeros(n_particles, 4).cuda()), shape_quats,
                 Variable(torch.zeros(count_nodes - n_particles - n_shapes, 4).cuda())], 0)
            state = torch.cat([positions, velocities, quats], 1)
        else:
            quat_null = np.array([[0., 0., 0., 0.]])
            quats = np.repeat(quat_null, [count_nodes], axis=0)
            quats[n_particles:n_particles + n_shapes] = shape_quats
            # if args.eval == 0:
            # quats += np.random.randn(quats.shape[0], 4) * 0.05
            state = torch.FloatTensor(np.concatenate([positions, velocities, quats], axis=1))
    else:
        if var:
            state = torch.cat([positions, velocities], 1)
        else:
            state = torch.FloatTensor(np.concatenate([positions, velocities], axis=1))

    if verbose:
        for i in range(count_nodes - 1):
            if np.sum(np.abs(attr[i] - attr[i + 1])) > 1e-6:
                print(i, attr[i], attr[i + 1])

        for i in range(len(Ras)):
            print(i, np.min(node_r_idxs[i]), np.max(node_r_idxs[i]), np.min(node_s_idxs[i]), np.max(node_s_idxs[i]))

    attr = torch.FloatTensor(attr)
    relations = [Rr_idxs, Rs_idxs, values, Ras, node_r_idxs, node_s_idxs, psteps]

    return attr, state, relations, n_particles, n_shapes, instance_idx

In [ ]:
phases_dict

In [ ]:
# generate data
datasets = {phase: PhysicsFleXDataset(
    args, phase, phases_dict, args.verbose_data) for phase in ['train', 'valid']}

In [ ]:
datasets['train'].load_data(args.env)

In [ ]:
idx = 0
idx_rollout = idx // (datasets['train'].args.time_step - 1)
idx_timestep = idx % (datasets['train'].args.time_step - 1)

# ignore the first frame for env RiceGrip
if datasets['train'].args.env == 'RiceGrip' and idx_timestep == 0:
    idx_timestep = np.random.randint(1, datasets['train'].args.time_step - 1)

data_path = os.path.join(datasets['train'].data_dir, str(idx_rollout), str(idx_timestep) + '.h5')
data_nxt_path = os.path.join(datasets['train'].data_dir, str(idx_rollout), str(idx_timestep + 1) + '.h5')

data = load_data(datasets['train'].data_names, data_path)

vel_his = []
for i in range(datasets['train'].args.n_his):
    path = os.path.join(datasets['train'].data_dir, str(idx_rollout), str(max(1, idx_timestep - i - 1)) + '.h5')
    data_his = load_data(datasets['train'].data_names, path)
    vel_his.append(data_his[1])

data[1] = np.concatenate([data[1]] + vel_his, 1)


In [ ]:
datasets['train'].stat

In [ ]:
data

In [ ]:
positions, velocities, hairs_idx = data
n_shapes = 1
hairs_idx_begin = [idx[0] for idx in hairs_idx]
hairs_idx_end = [idx[1] for idx in hairs_idx]

clusters = None

n_particles = positions.size


In [ ]:
positions[:3,0:3]

In [ ]:
instance_idx = [0, n_particles]
### object attributes
count_nodes = n_particles
attr = np.zeros((count_nodes, args.attr_dim))

In [ ]:

##### add env specific graph components
rels = []

In [ ]:
[1]*2

In [ ]:
attr[:n_particles,0] = 1
#rels += find_relations_neighbor(pos, queries, anchors, args.neighbor_radius, 2, var)
#queries = set of hairs, anchors = stick
for i in range(n_particles):
    if i in hairs_idx_begin:
        receiver = [i+1]
        sender = [i]
        relation_type = [1]
    elif i in hairs_idx_end:
        receiver = [i-1]
        sender = [i]
        relation_type = [1]
    else:
        receiver = [i-1,i+1]
        sender = [i, i]
        relation_type = [1,1]
    rels.append(np.stack([receiver, sender, relation_type], axis=1))
    

In [ ]:
rels

In [ ]:
rels = np.concatenate(rels, 0)

In [ ]:
### construct relations
Rr_idxs = []        # relation receiver idx list
Rs_idxs = []        # relation sender idx list
Ras = []            # relation attributes list
values = []         # relation value list (should be 1)
node_r_idxs = []    # list of corresponding receiver node idx
node_s_idxs = []    # list of corresponding sender node idx
psteps = []         # propagation steps


Rr_idxs.append(torch.LongTensor([rels[:, 0], np.arange(rels.shape[0])]))
Rs_idxs.append(torch.LongTensor([rels[:, 1], np.arange(rels.shape[0])]))
Ra = np.zeros((rels.shape[0], args.relation_dim))
Ras.append(torch.FloatTensor(Ra))
values.append(torch.FloatTensor([1] * rels.shape[0]))
node_r_idxs.append(np.arange(n_particles))
node_s_idxs.append(np.arange(n_particles + n_shapes))
psteps.append(args.pstep)

In [ ]:
data = [positions, velocities]
stat = datasets['train'].stat
data = normalize(data, stat)
positions, velocities = data[0], data[1]

In [ ]:
attr = torch.FloatTensor(attr)
relations = [Rr_idxs, Rs_idxs, values, Ras, node_r_idxs, node_s_idxs, psteps]

In [ ]:
relations

In [ ]:
def prepare_input(data, stat, args, phases_dict, verbose=0, var=False):
    
    positions, velocities, hairs_idx = data
    n_shapes = 1
    hairs_idx_begin = [idx[0] for idx in hairs_idx]
    clusters = None

    count_nodes = positions.size(0) if var else positions.shape[0]
    n_particles = count_nodes - n_shapes

    ### instance idx
    #   instance_idx (n_instance + 1): start idx of instance
    if args.env == 'RiceGrip' or args.env == 'FluidShake':
        instance_idx = [0, n_particles]
    elif args.env == 'Hairs':
        instance_idx = [0, count_nodes]
    else:
        instance_idx = phases_dict["instance_idx"]
    if verbose:
        print("Instance_idx:", instance_idx)


    ### object attributes
    #   dim 10: [rigid, fluid, root_0, root_1, gripper_0, gripper_1, mass_inv,
    #            clusterStiffness, clusterPlasticThreshold, cluasterPlasticCreep]
    attr = np.zeros((count_nodes, args.attr_dim))
    # no need to include mass for now
    # attr[:, 6] = positions[:, -1].data.cpu().numpy() if var else positions[:, -1] # mass_inv
    if args.env == 'RiceGrip':
        # clusterStiffness, clusterPlasticThreshold, cluasterPlasticCreep
        attr[:, -3:] = scene_params[-3:]


    ### construct relations
    Rr_idxs = []        # relation receiver idx list
    Rs_idxs = []        # relation sender idx list
    Ras = []            # relation attributes list
    values = []         # relation value list (should be 1)
    node_r_idxs = []    # list of corresponding receiver node idx
    node_s_idxs = []    # list of corresponding sender node idx
    psteps = []         # propagation steps

    ##### add env specific graph components
    rels = []
    if args.env == 'RiceGrip':
        # nodes = np.arange(n_particles)
        for i in range(n_shapes):
            attr[n_particles + i, 2 + i] = 1

            pos = positions.data.cpu().numpy() if var else positions
            dis = np.linalg.norm(
                pos[:n_particles, 3:6:2] - pos[n_particles + i, 3:6:2], axis=1)
            nodes = np.nonzero(dis < 0.3)[0]

            if verbose:
                visualize_neighbors(positions, positions, 0, nodes)
                print(np.sort(dis)[:10])

            gripper = np.ones(nodes.shape[0], dtype=np.int) * (n_particles + i)
            rels += [np.stack([nodes, gripper, np.ones(nodes.shape[0])], axis=1)]
            
    elif args.env = 'Hairs':
        # TODO: add relations between the hairs and the stick
        attr[:,0] = 1
        pass

    elif args.env == 'FluidShake':
        for i in range(n_shapes):
            attr[n_particles + i, 1 + i] = 1

            pos = positions.data.cpu().numpy() if var else positions
            if i == 0:
                # floor
                dis = pos[:n_particles, 1] - pos[n_particles + i, 1]
            elif i == 1:
                # left
                dis = pos[:n_particles, 0] - pos[n_particles + i, 0]
            elif i == 2:
                # right
                dis = pos[n_particles + i, 0] - pos[:n_particles, 0]
            elif i == 3:
                # back
                dis = pos[:n_particles, 2] - pos[n_particles + i, 2]
            elif i == 4:
                # front
                dis = pos[n_particles + i, 2] - pos[:n_particles, 2]
            else:
                raise AssertionError("more shapes than expected")
            nodes = np.nonzero(dis < 0.1)[0]

            if verbose:
                visualize_neighbors(positions, positions, 0, nodes)
                print(np.sort(dis)[:10])

            wall = np.ones(nodes.shape[0], dtype=np.int) * (n_particles + i)
            rels += [np.stack([nodes, wall, np.ones(nodes.shape[0])], axis=1)]

    if verbose and len(rels) > 0:
        print(np.concatenate(rels, 0).shape)

    ##### add relations between leaf particles
    for i in range(len(instance_idx) - 1):
        st, ed = instance_idx[i], instance_idx[i + 1]

        if verbose:
            print('instance #%d' % i, st, ed)

        if args.env == 'BoxBath':
            if phases_dict['material'][i] == 'rigid':
                attr[st:ed, 0] = 1
                queries = np.arange(st, ed)
                anchors = np.concatenate((np.arange(st), np.arange(ed, n_particles)))
            elif phases_dict['material'][i] == 'fluid':
                attr[st:ed, 1] = 1
                queries = np.arange(st, ed)
                anchors = np.arange(n_particles)
            else:
                raise AssertionError("Unsupported materials")
                
        elif args.env == 'Hairs':
            # TODO: add relations between the hairs and the stick
            pass
            
          #  if ed not in hairs_idx_begin:
             #   queries = np.arange(st, ed)
              #  anchors = np.arange(n_particles)

        elif args.env == 'FluidFall' or args.env == 'RiceGrip' or args.env == 'FluidShake':
            if phases_dict['material'][i] == 'fluid':
                attr[st:ed, 0] = 1
                queries = np.arange(st, ed)
                anchors = np.arange(n_particles)
            else:
                raise AssertionError("Unsupported materials")

        else:
            raise AssertionError("Unsupported materials")

        # st_time = time.time()
        pos = positions
        pos = pos[:, -3:]
        if args.env == 'Hairs':
            #TODO
            pass
        else:
            rels += find_relations_neighbor(pos, queries, anchors, args.neighbor_radius, 2, var)
            # return list of [receiver, sender, relation_type]
        # print("Time on neighbor search", time.time() - st_time)

    if verbose:
        print("Attr shape (after add env specific graph components):", attr.shape)
        print("Object attr:", np.sum(attr, axis=0))

    rels = np.concatenate(rels, 0)
    if rels.shape[0] > 0:
        if verbose:
            print("Relations neighbor", rels.shape)
        Rr_idxs.append(torch.LongTensor([rels[:, 0], np.arange(rels.shape[0])]))
        Rs_idxs.append(torch.LongTensor([rels[:, 1], np.arange(rels.shape[0])]))
        Ra = np.zeros((rels.shape[0], args.relation_dim))
        Ras.append(torch.FloatTensor(Ra))
        values.append(torch.FloatTensor([1] * rels.shape[0]))
        node_r_idxs.append(np.arange(n_particles))
        node_s_idxs.append(np.arange(n_particles + n_shapes))
        psteps.append(args.pstep)

    if verbose:
        print('clusters', clusters)

    # add heirarchical relations per instance
    cnt_clusters = 0
    for i in range(len(instance_idx) - 1):
        st, ed = instance_idx[i], instance_idx[i + 1]
        n_root_level = len(phases_dict["root_num"][i])

        if n_root_level > 0:
            attr, positions, velocities, count_nodes, \
            rels, node_r_idx, node_s_idx, pstep = \
                    make_hierarchy(args.env, attr, positions, velocities, i, st, ed,
                                   phases_dict, count_nodes, clusters[cnt_clusters], verbose, var)

            for j in range(len(rels)):
                if verbose:
                    print("Relation instance", j, rels[j].shape)
                Rr_idxs.append(torch.LongTensor([rels[j][:, 0], np.arange(rels[j].shape[0])]))
                Rs_idxs.append(torch.LongTensor([rels[j][:, 1], np.arange(rels[j].shape[0])]))
                Ra = np.zeros((rels[j].shape[0], args.relation_dim)); Ra[:, 0] = 1
                Ras.append(torch.FloatTensor(Ra))
                values.append(torch.FloatTensor([1] * rels[j].shape[0]))
                node_r_idxs.append(node_r_idx[j])
                node_s_idxs.append(node_s_idx[j])
                psteps.append(pstep[j])

            cnt_clusters += 1

    if verbose:
        if args.env == 'RiceGrip' or args.env == 'FluidShake':
            print("Scene_params:", scene_params)

        print("Attr shape (after hierarchy building):", attr.shape)
        print("Object attr:", np.sum(attr, axis=0))
        print("Particle attr:", np.sum(attr[:n_particles], axis=0))
        print("Shape attr:", np.sum(attr[n_particles:n_particles+n_shapes], axis=0))
        print("Roots attr:", np.sum(attr[n_particles+n_shapes:], axis=0))

    ### normalize data
    data = [positions, velocities]
    data = normalize(data, stat, var)
    positions, velocities = data[0], data[1]

    if verbose:
        print("Particle positions stats")
        print(positions.shape)
        print(np.min(positions[:n_particles], 0))
        print(np.max(positions[:n_particles], 0))
        print(np.mean(positions[:n_particles], 0))
        print(np.std(positions[:n_particles], 0))

        show_vel_dim = 6 if args.env == 'RiceGrip' else 3
        print("Velocities stats")
        print(velocities.shape)
        print(np.mean(velocities[:n_particles, :show_vel_dim], 0))
        print(np.std(velocities[:n_particles, :show_vel_dim], 0))

    if args.env == 'RiceGrip':
        if var:
            quats = torch.cat(
                [Variable(torch.zeros(n_particles, 4).cuda()), shape_quats,
                 Variable(torch.zeros(count_nodes - n_particles - n_shapes, 4).cuda())], 0)
            state = torch.cat([positions, velocities, quats], 1)
        else:
            quat_null = np.array([[0., 0., 0., 0.]])
            quats = np.repeat(quat_null, [count_nodes], axis=0)
            quats[n_particles:n_particles + n_shapes] = shape_quats
            # if args.eval == 0:
            # quats += np.random.randn(quats.shape[0], 4) * 0.05
            state = torch.FloatTensor(np.concatenate([positions, velocities, quats], axis=1))
    else:
        if var:
            state = torch.cat([positions, velocities], 1)
        else:
            state = torch.FloatTensor(np.concatenate([positions, velocities], axis=1))

    if verbose:
        for i in range(count_nodes - 1):
            if np.sum(np.abs(attr[i] - attr[i + 1])) > 1e-6:
                print(i, attr[i], attr[i + 1])

        for i in range(len(Ras)):
            print(i, np.min(node_r_idxs[i]), np.max(node_r_idxs[i]), np.min(node_s_idxs[i]), np.max(node_s_idxs[i]))

    attr = torch.FloatTensor(attr)
    relations = [Rr_idxs, Rs_idxs, values, Ras, node_r_idxs, node_s_idxs, psteps]

    return attr, state, relations, n_particles, n_shapes, instance_idx

In [ ]:

attr, state, relations, n_particles, n_shapes, instance_idx = \
        prepare_input(data, self.stat, self.args, self.phases_dict, self.verbose)

### label
data_nxt = normalize(load_data(self.data_names, data_nxt_path), self.stat)

label = torch.FloatTensor(data_nxt[1][:n_particles])

In [ ]:
data = datasets['train'].__getitem__(1)

In [ ]:
print(data)